# Topic of the Project

Sentimental Analysis of Employee Review

# Goal Of the Project

The main reason for choosing this dataset was to get an insight on why employees from big companies like Google, Amazon, Microsoft are refraining from working there, is it their working culture or politics inside the company or the way they treat their employees. Through some detailed insights about the company from their recent or current employees, it can help the new joiner to decide whether is it the right place to start their career with.

# Target Audiences

This dataset can be useful for both, The Organization and the User.
 
From Organizational Point of View :-
Predicting what are the chances that the current employee will either stay/leave the organisation.
To improve the organisation’s working culture or work-life balance, such reviews and the results will help them to spot a place for improvement.
Employee Feedback are important for the organisation.
 
From User Point of View :-
It can help the new joiner to decide whether is it the right place to start their career with.


# Importing Libraries

In [1]:
import pandas as pd
import numpy as np  
import re  
import nltk  
from sklearn.datasets import load_files  
nltk.download('stopwords')  
import pickle  
import datetime
from datetime import datetime
from nltk.corpus import stopwords 
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
import seaborn as sns
import plotly 
import plotly.plotly as py
import matplotlib.gridspec as gridspec
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='sanket.bhaud', api_key='TUPydu3LQOjGVAnHZzPw')
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sanket/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/sanket/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv("/Users/sanket/Documents/Projects/SML/employee_reviews.csv")

This dataset contains employee reviews separated into the following categories:

1. Index: index
2. Company: Company name
3. Location : This includes the country's name in parenthesis [i.e "Toronto,(Canada)"].
4. Job-Title: This string will also include whether the reviewer is a 'Current' or 'Former' Employee at the time of the review
5. Summary: Short summary of employee review
6. Pros: Pros
7. Cons: Cons
8. Overall Rating: 1-5
9. Work/Life Balance Rating: 1-5
10. Culture and Values Rating: 1-5
11. Career Opportunities Rating: 1-5
12. Comp & Benefits Rating: 1-5
13. Senior Management Rating: 1-5
14. Helpful Review Count: A count of how many people found the review to be helpful
15. Link to Review : This will provide you with a direct link to the page that contains the review. However it is likely that this link will be outdated

In [3]:
df.shape

(67525, 17)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67525 entries, 0 to 67524
Data columns (total 17 columns):
Unnamed: 0                    67525 non-null int64
company                       67525 non-null object
location                      67525 non-null object
dates                         67525 non-null object
job-title                     67525 non-null object
summary                       67405 non-null object
pros                          67525 non-null object
cons                          67525 non-null object
advice-to-mgmt                67228 non-null object
overall-ratings               67525 non-null int64
work-balance-stars            67525 non-null object
culture-values-stars          67525 non-null object
carrer-opportunities-stars    67525 non-null object
comp-benefit-stars            67525 non-null object
senior-mangemnet-stars        67525 non-null object
helpful-count                 67525 non-null int64
link                          67525 non-null object
dtypes: in

In [5]:
#Droppig columns
df.drop(columns=['location', 'link', 'helpful-count', 'Unnamed: 0'], inplace=True)

In [6]:
df.head()

,company,dates,job-title,summary,pros,cons,advice-to-mgmt,overall-ratings,work-balance-stars,culture-values-stars,carrer-opportunities-stars,comp-benefit-stars,senior-mangemnet-stars
0,google,"Dec 11, 2018",Current Employee - Anonymous Employee,Best Company to work for,People are smart and friendly,Bureaucracy is slowing things down,none,5,4,5,5,4,5
1,google,"Jun 21, 2013",Former Employee - Program Manager,"Moving at the speed of light, burn out is inev...","1) Food, food, food. 15+ cafes on main campus ...",1) Work/life balance. What balance? All those ...,1) Don't dismiss emotional intelligence and ad...,4,2,3,3,5,3
2,google,"May 10, 2014",Current Employee - Software Engineer III,Great balance between big-company security and...,"* If you're a software engineer, you're among ...","* It *is* becoming larger, and with it comes g...",Keep the focus on the user. Everything else wi...,5,5,4,5,5,4
3,google,"Feb 8, 2015",Current Employee - Anonymous Employee,The best place I've worked and also the most d...,You can't find a more well-regarded company th...,I live in SF so the commute can take between 1...,Keep on NOT micromanaging - that is a huge ben...,5,2,5,5,4,5
4,google,"Jul 19, 2018",Former Employee - Software Engineer,"Unique, one of a kind dream job",Google is a world of its own. At every other c...,"If you don't work in MTV (HQ), you will be giv...",Promote managers into management for their man...,5,5,5,5,5,5


# Cleaning Process

In [7]:
df['dates'] = df['dates'].apply(lambda x: datetime.strptime(x, ' %b %d, %Y'))

In [8]:
new = df["job-title"].str.split(" - ", n = 1, expand = True)
df["job_type"] = new[0]
df["job_title"] = new[1]

In [9]:
df.head()

,company,dates,job-title,summary,pros,cons,advice-to-mgmt,overall-ratings,work-balance-stars,culture-values-stars,carrer-opportunities-stars,comp-benefit-stars,senior-mangemnet-stars,job_type,job_title
0,google,2018-12-11,Current Employee - Anonymous Employee,Best Company to work for,People are smart and friendly,Bureaucracy is slowing things down,none,5,4,5,5,4,5,Current Employee,Anonymous Employee
1,google,2013-06-21,Former Employee - Program Manager,"Moving at the speed of light, burn out is inev...","1) Food, food, food. 15+ cafes on main campus ...",1) Work/life balance. What balance? All those ...,1) Don't dismiss emotional intelligence and ad...,4,2,3,3,5,3,Former Employee,Program Manager
2,google,2014-05-10,Current Employee - Software Engineer III,Great balance between big-company security and...,"* If you're a software engineer, you're among ...","* It *is* becoming larger, and with it comes g...",Keep the focus on the user. Everything else wi...,5,5,4,5,5,4,Current Employee,Software Engineer III
3,google,2015-02-08,Current Employee - Anonymous Employee,The best place I've worked and also the most d...,You can't find a more well-regarded company th...,I live in SF so the commute can take between 1...,Keep on NOT micromanaging - that is a huge ben...,5,2,5,5,4,5,Current Employee,Anonymous Employee
4,google,2018-07-19,Former Employee - Software Engineer,"Unique, one of a kind dream job",Google is a world of its own. At every other c...,"If you don't work in MTV (HQ), you will be giv...",Promote managers into management for their man...,5,5,5,5,5,5,Former Employee,Software Engineer


In [10]:
#Df Transformations
df['work-balance-stars'] = np.where(df['work-balance-stars']=='none', 0, df['work-balance-stars'])
df['culture-values-stars'] = np.where(df['culture-values-stars']=='none', 0, df['culture-values-stars'])
df['carrer-opportunities-stars'] = np.where(df['carrer-opportunities-stars']=='none', 0, df['carrer-opportunities-stars'])
df['comp-benefit-stars'] = np.where(df['comp-benefit-stars']=='none', 0, df['comp-benefit-stars'])
df['senior-mangemnet-stars'] = np.where(df['senior-mangemnet-stars']=='none', 0, df['senior-mangemnet-stars'])
df['work-balance-stars'] = pd.to_numeric(df['work-balance-stars'])
df['culture-values-stars'] = pd.to_numeric(df['culture-values-stars'])
df['carrer-opportunities-stars'] = pd.to_numeric(df['carrer-opportunities-stars'])
df['comp-benefit-stars'] = pd.to_numeric(df['comp-benefit-stars'])
df['senior-mangemnet-stars'] = pd.to_numeric(df['senior-mangemnet-stars'])

# NLP Cleaning Process

In [11]:
def clean_data(data,column_name):
    documents = []
    document1 = ''

    stemmer = WordNetLemmatizer()

    for sen in range(0, len(data)):  
        # Remove all the special characters
        document = re.sub(r'\W', ' ', str(data[sen]))

        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 

        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)
    
        document = re.sub(r'[0-9]+',"", document)

        # Converting to Lowercase
        document = document.lower()

        # Lemmatization
        document = document.split()

        document = [stemmer.lemmatize(word) for word in document]
        document = ' '.join(document)
    
        documents.append(document)
        df.at[sen,column_name] = document
    


In [12]:
clean_data(df['pros'],'pros')

In [13]:
clean_data(df['cons'],'cons')

In [14]:
clean_data(df['summary'],'summary')

In [15]:
clean_data(df['advice-to-mgmt'],'advice-to-mgmt')

# Visualization 1

In [16]:
#Plotting the number of reviews present for each company
counts = df['company'].value_counts()
counts = pd.DataFrame(counts)
trace1 = go.Bar(
                x = counts.index,
                y = counts.company,
                name = "Number of reviews for each company",
                marker = dict(color='rgb(26, 118, 255)',
                             line=dict(color='black',width=1.5)),
                text = counts.company)
data = [trace1]
layout = go.Layout(barmode = "group",title='Number of reviews of each company')
fig = go.Figure(data = data, layout = layout)
py.iplot(fig)

# Visualization 2

In [17]:
def find_employee_type(s):
    x = re.search(r'([^-]*)-',s).group(1)
    return str(x)
    
df['employee-type'] = df['job-title'].apply(find_employee_type)
cmp_lc= df[["company","employee-type"]] 
cmp_lc = cmp_lc.groupby(["company", "employee-type"]).size().reset_index()
cmp_lc = cmp_lc.rename(columns={0: 'total_reviews_by_title'})

years = list(cmp_lc.company.unique())
company_data = []
v = True

for i in years:
    if i!='amazon':
        v=False
    data_upd = [dict(type='bar',
                     visible = v,
                     x = cmp_lc[cmp_lc['company']==i]['employee-type'],
                     y = cmp_lc[cmp_lc['company']==i]['total_reviews_by_title'],
                     textposition = 'auto',
                     marker=dict(
                     color='rgb(158,202,225)',
                     line=dict(
                         color='rgb(8,48,107)',
                         width=1.5),
                     ),
                 opacity=0.6)]
    
    company_data.extend(data_upd)

years = [x.capitalize() for x in years]

# set menus inside the plot
steps = []
yr = 0
for i in range(0,len(company_data)):
    step = dict(method = "restyle",
                args = ["visible", [False]*len(company_data)],
                label = years[yr]) 
    step['args'][1][i] = True
    steps.append(step)
    yr += 1
    

sliders = [dict(active = 6,
                currentvalue = {"prefix": "Company: "},
                pad = {"t": 50},
                steps = steps)]

# Set the layout
layout = dict(title = 'Do current employees review the comapnies more than the ex-employees?',
              sliders = sliders)

fig = dict(data=company_data, layout=layout)
py.iplot(fig)

# Visualization 3

In [18]:
df['year'] = pd.DatetimeIndex(df.dates).year  

In [20]:
cmp_lc = df[["company","dates"]]
cmp_lc = cmp_lc.reset_index(drop=True)
cmp_lc.sort_values(by='dates')
cmp_lc = cmp_lc.groupby(["company", "dates"]).size().reset_index()
cmp_lc = cmp_lc.rename(columns={0: 'total_reviews'})

amazon = go.Scatter(
    x=cmp_lc[cmp_lc['company']=='amazon']['dates'],
    y=cmp_lc[cmp_lc['company']=='amazon']['total_reviews'],
    name = "Amazon",
    line = dict(color = '#17BECF'),
    opacity = 0.8)

microsoft = go.Scatter(
    x=cmp_lc[cmp_lc['company']=='microsoft']['dates'],
    y=cmp_lc[cmp_lc['company']=='microsoft']['total_reviews'],
    name = "Microsoft",
    line = dict(color = '#7F7F7F'),
    opacity = 0.8)

apple = go.Scatter(
    x=cmp_lc[cmp_lc['company']=='apple']['dates'],
    y=cmp_lc[cmp_lc['company']=='apple']['total_reviews'],
    name = "Apple",
    line = dict(color = '#0fac1f'),
    opacity = 0.8)

google = go.Scatter(
    x=cmp_lc[cmp_lc['company']=='google']['dates'],
    y=cmp_lc[cmp_lc['company']=='google']['total_reviews'],
    name = "Google",
    line = dict(color = '#6335c3'),
    opacity = 0.8)

facebook = go.Scatter(
    x=cmp_lc[cmp_lc['company']=='facebook']['dates'],
    y=cmp_lc[cmp_lc['company']=='facebook']['total_reviews'],
    name = "Facebook",
    line = dict(color = '#808000'),
    opacity = 0.8)


data = [amazon,microsoft,apple,google,facebook]

layout = dict(
    title='Analysing any trend between number of reviews and dates',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date'
    )
)

fig = dict(data=data, layout=layout)
py.iplot(fig)

# Visualization 4


In [21]:
from textblob import TextBlob
df[['summary']] = df[['summary']].astype(str)
df['polarity'] = df.apply(lambda x: TextBlob(x['summary']).sentiment.polarity, axis=1)

 
def sentiment(df):
    if df['polarity'] > 0:
        val = "Positive"
    elif df['polarity'] == 0:
        val = "Neutral"
    else:
        val = "Negative"
    return val

df['sentiment_type'] = df.apply(sentiment, axis=1)

cmp_lc = df[["company","sentiment_type"]] 
cmp_lc = cmp_lc.groupby(["company", "sentiment_type"]).size().reset_index()
cmp_lc = cmp_lc.rename(columns={0: 'total_sentiment_types'})

#For plotting it
years = list(cmp_lc.company.unique())
company_data = []
v = True

for i in years:
    if i!='amazon':
        v=False
    data_upd = [dict(type='bar',
                     visible = v,
                     x = cmp_lc[cmp_lc['company']==i]['sentiment_type'],
                     y = cmp_lc[cmp_lc['company']==i]['total_sentiment_types'],
                     textposition = 'auto',
                     marker=dict(
                     color='orange',
                     line=dict(
                         width=1.5),
                     ),
                 opacity=0.6)]
    
    company_data.extend(data_upd)

years = [x.capitalize() for x in years]

# set menus inside the plot
steps = []
yr = 0
for i in range(0,len(company_data)):
    step = dict(method = "restyle",
                args = ["visible", [False]*len(company_data)],
                label = years[yr]) 
    step['args'][1][i] = True
    steps.append(step)
    yr += 1
    

sliders = [dict(active = 6,
                currentvalue = {"prefix": "Company: "},
                pad = {"t": 50},
                steps = steps)]

# Set the layout
layout = dict(title = 'Sentiment analysis for summary text data for each company',
              sliders = sliders)
fig = dict(data=company_data, layout=layout)
py.iplot(fig)